In [251]:
import numpy as np
from tabulate import tabulate

In [252]:
criteria = np.loadtxt('task_matrix.txt', delimiter=' ', dtype=int)
criteria

array([[10,  3, 10, 10,  4,  3,  4,  1,  8,  3,  8,  4],
       [ 8,  3,  8,  7,  2,  3,  4,  1,  8,  3,  8,  4],
       [10,  8, 10, 10,  4,  9,  4,  3,  8,  4,  8,  7],
       [10,  8, 10, 10,  5,  9,  8,  4,  8,  5,  8,  7],
       [ 5,  3,  4,  6,  2,  3,  4,  1,  1,  3,  1,  4],
       [ 3,  2,  4,  5,  2,  3,  4,  1,  1,  2,  1,  4],
       [10,  8, 10, 10,  7,  9,  4,  8,  8,  5,  8,  7],
       [10,  8, 10, 10,  7,  9,  4,  8,  8,  7,  8,  7],
       [10,  8, 10, 10,  7,  9,  9,  8, 10, 10,  8,  7],
       [ 1,  4,  2,  5,  7,  8,  9,  8,  1,  1,  2,  7],
       [ 1,  2,  2,  5,  2,  3,  4,  1,  1,  1,  1,  4],
       [ 1,  2,  2,  1,  2,  3,  4,  1,  1,  1,  1,  4],
       [10,  8, 10, 10,  7,  9,  9,  8, 10, 10,  8, 10],
       [10,  9, 10, 10,  7,  9,  9,  8, 10, 10,  8, 10],
       [ 2,  7, 10,  1,  7,  1,  4,  8,  5,  7,  6,  7],
       [ 2,  7, 10,  1,  7,  3,  4,  8,  5,  9,  6,  8],
       [10,  9, 10, 10, 10,  4,  4,  8,  8,  9,  8,  8],
       [10,  9, 10, 10, 10,  4,

#### Формуємо множину веторів сигма.

In [253]:
def make_delta_vector(criteria, index_row): #утворення множини векторів дельта, щоб потім використати для знаходження сигм
    deltas = []
    for i in range(criteria.shape[0]):
        deltas.append(criteria[index_row]-criteria[i])
    deltas = np.array(deltas)
    return deltas
    
def sigma_vectors(criteria):
    sigma_vect = []
    
    for i in range(criteria.shape[0]):
        delta = make_delta_vector(criteria, i)
        
        delta_positive = np.where(delta > 0, 1, 0)
        delta_negative = np.where(delta < 0, -1, 0)

        sigma = delta_positive+delta_negative
        sigma_vect.append(sigma)
        
    sigma_vect = np.array(sigma_vect)
    return sigma_vect

In [254]:
sigma_vect = sigma_vectors(criteria)
sigma_vect

array([[[ 0,  0,  0, ...,  0,  0,  0],
        [ 1,  0,  1, ...,  0,  0,  0],
        [ 0, -1,  0, ..., -1,  0, -1],
        ...,
        [ 0, -1,  0, ..., -1,  0, -1],
        [ 0, -1,  0, ..., -1,  0, -1],
        [ 1, -1,  1, ..., -1,  1,  1]],

       [[-1,  0, -1, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [-1, -1, -1, ..., -1,  0, -1],
        ...,
        [-1, -1, -1, ..., -1,  0, -1],
        [-1, -1, -1, ..., -1,  0, -1],
        [ 1, -1,  1, ..., -1,  1,  1]],

       [[ 0,  1,  0, ...,  1,  0,  1],
        [ 1,  1,  1, ...,  1,  0,  1],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0, -1,  0, ..., -1,  0, -1],
        [ 0, -1,  0, ..., -1,  0, -1],
        [ 1, -1,  1, ...,  0,  1,  1]],

       ...,

       [[ 0,  1,  0, ...,  1,  0,  1],
        [ 1,  1,  1, ...,  1,  0,  1],
        [ 0,  1,  0, ...,  1,  0,  1],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0, -1,  0, ...,  0,  0, -1],
        [ 1, -1,  1, ...,  1,  1

#### Реалізація функцій для для оптимізації за домінуванням і блокуванням

In [255]:
def I(R): #формуємо симетричну частину
    return (R == R.T)*R

def P(R):#формуємо асиметричну частину
    return R-I(R)

def N(R):#формуємо відношення непорівнюваності
    return (R == R.T)-I(R)

def check_asymmetry(R):#перевірка на асиметричність. True - якщо асиметричне
    return not I(R).any()

#Оптимізація за домінуванням
def dom_opt(R, m):
    opt_res = {
        'l_p': [],
        'l_r': [],
        'strong_l_r': []}
    
    if check_asymmetry(R):
        opt_res['l_p'] = largest_p(R, m)
    else:
        opt_res['l_r'], opt_res['strong_l_r'] = largest_r(R, m)
    return opt_res

def largest_p(R, m):#найбільші по P
    for row, i in zip(R, range(m)):
        if row.sum()==m-1 and row[i]==0:
            return [i+1] #бо одночасно 2 таких елементи існувати не може
    return []

def largest_r(R, m):#найбільші та строго найбільші по R 
    lr, strong_lr = [], []
    for row, i in zip(R, range(m)):
        if row.sum()==m:
            lr.append(i+1)
            if R.T[i].sum()==1:
                strong_lr.append(i+1)
    return lr, strong_lr
            
#Оптимізація за блокуванням
def block_opt(R, m):
    opt_res = {
        'm_p': [],
        'm_r': [],
        'strong_m_r': []}
        
    if check_asymmetry(R):
        opt_res['m_p'] = max_p(R, m)
    else:
        opt_res['m_r'], opt_res['strong_m_r'] = max_r(R, m)
    return opt_res

def max_p(R, m):#максимальні по P
    mp = []
    for row, i in zip(R.T, range(m)):
        if not row.any():
            mp.append(i+1)
    return mp

def max_r(R, m):#максимальні та строго максимальні по R
    mr, strong_mr = [], []
    for row, i in zip(P(R).T, range(m)):
        if not row.any():
            mr.append(i+1)
            if R.T[i].sum()==0 or (R.T[i].sum()==1 and R.T[i][i] == 1):
                strong_mr.append(i+1)
    return mr, strong_mr

def optimization(R):
    m = R.shape[0]
    headers = ["Множини альтернатив","Оптимальний розв'язок"]
    
    solution = dom_opt(R, m)
    if len(solution['l_p']) or len(solution['l_r']) or len(solution['strong_l_r']):
        if len(solution['l_p']) == 0:
            del solution['l_p']
        else:
            del solution['l_r']
            del solution['strong_l_r']
        print(tabulate(solution.items(), headers = headers))
    else:
        solution = block_opt(R, m)
        if len(solution['m_p']) == 0:
            del solution['m_p']
        else:
            del solution['m_r']
            del solution['strong_m_r']
        print(tabulate(solution.items(), headers = headers))

#### Відношення Парето

In [272]:
def pareto(sigma_vect):
    m = sigma_vect.shape[0] #кількість x
    solution = np.zeros((m, m), dtype=int)
    
    for row in range(m):
        for col in range(m):
            if (sigma_vect[row][col] >= 0).all():
                solution[row][col] = 1
    
    return solution

In [273]:
pareto_mtrx = pareto(sigma_vect)
print("Відношення Парето")
pareto_mtrx

Відношення Парето


array([[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [282]:
print("ВІДНОШЕННЯ ПАРЕТО\n____________________________________")
optimization(pareto_mtrx)

ВІДНОШЕННЯ ПАРЕТО
____________________________________
Множини альтернатив    Оптимальний розв'язок
---------------------  -----------------------
m_r                    [14, 19]
strong_m_r             [14, 19]


#### Мажоритарне відношення

In [276]:
def majority(sigma_vect):
    m = sigma_vect.shape[0] #кількість x
    sigma_sum = np.array([[sigma_vect[i][j].sum() for j in range(m)] for i in range(m)])
    Pm = np.where(sigma_sum > 0, 1, 0)
    return Pm

In [278]:
majority_mtrx = majority(sigma_vect)
majority_mtrx

array([[0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 0, 1, 1, 0, 0, 0,

In [283]:
print("МАЖОРИТАРНЕ ВІДНОШЕННЯ\n____________________________________")
optimization(majority_mtrx)

МАЖОРИТАРНЕ ВІДНОШЕННЯ
____________________________________
Множини альтернатив    Оптимальний розв'язок
---------------------  -----------------------
l_p                    [14]


#### Відношення лексикографії